In [3]:
from pylab import *


def slice_img(baseImg, detailSize):
    flattenedLength = baseImg.shape[0] - detailSize
    flattenedWidth = baseImg.shape[1] - detailSize
    details = np.zeros((flattenedLength * flattenedWidth, detailSize ** 2))

    for i in range(flattenedLength):
        for j in range(flattenedWidth):
            detail = baseImg[i:i + detailSize, j:j + detailSize].flatten()
            details[i * j, :] = detail

    return details


def sorted_eig(M):
    d, U = eig(M)
    si = argsort(d)[-1::-1]
    d = d[si]
    U = U[:,si]
    return (d,U)


def show_detail(details, detailSize, x, y):
    detailFlat = details[x * y, :]
    detail = detailFlat.reshape((detailSize, detailSize))
    return detail


def remove_dimensions(details, det, U, k):
    mean = np.mean(details, axis=0)
    xzm = det.flatten() - mean
    yzm = np.dot(np.transpose(U), xzm)

    yzm_k = yzm[:k]
    xzm_k = np.dot(U[:,:k], yzm_k)
    x_k = xzm_k + mean
    return x_k


def main():
    baseImg = imread('data/trui.png')
    detailSize = 25
    details = slice_img(baseImg, detailSize)

    cov = np.cov(details, rowvar=False)

    d, U = sorted_eig(cov)


    # Image
    plt.subplot(2, 3, 1)
    plt.title('Base image')
    plt.imshow(baseImg)

    # Scree diagram
    plt.subplot(2,3,2)
    plt.title('Scree diagram [0, 100]')
    xd = np.arange(0, len(d), 1)
    plt.bar(xd, d)
    plt.xlim(0, 100)

    # Covalence plot
    plt.subplot(2,3,3)
    plt.title('Covalence plot')
    plt.imshow(cov)

    # Detail at 0,0
    plt.subplot(2,3,4)
    plt.title('Detail')
    det = show_detail(details, detailSize, 125, 125)
    plt.imshow(det)

    # Detail with k=15
    plt.subplot(2,3,5)
    plt.title('Detail using k=15 eigenvectors')
    det_k = remove_dimensions(details, det, U, 15).reshape((detailSize, detailSize))
    plt.imshow(det_k)

    # Detail with k=30
    """ k=30 seems to result in a sufficiently accurate result """
    plt.subplot(2,3,6)
    plt.title('Detail using k=30 eigenvectors')
    det_k = remove_dimensions(details, det, U, 30).reshape((detailSize, detailSize))
    plt.imshow(det_k)

    plt.show()


if __name__ == '__main__':
    main()